# Problemas multiclase

Hasta ahora sólo hemos atacado el problema de clasificación binaria. 
Pero frecuentemente nos encontraremos con conjuntos de datos donde hay varias clases y varias etiquetas
+ multiclase: los ejemplos del conjunto de datos se agrupan en un conjunto finito de clases diferentes.
+ multietiqueta: cada ejemplo puede tener una o varias etiquetas.

En este cuaderno vamos a ver algunos métodos para abordad este tipo de problemas. 

---
    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2021, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2021, URJC (Spain)

    alfredo.cuesta@urjc.es

#### Preliminares

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
sys.path.append('../MyUtils/')
import MyUtils as my
seed = 1234 #<- random generator seed (comment to get randomness)

#### Cargar datos desde OpenML a dataframes
+ [OpenML](https://www.openml.org/d) es un repositorio con miles de conjuntos de datos
+ Scikit-learn es capaz de descargarse un conjunto dado con el método `fetch_openml`

In [2]:
# Load MNIST data from OpenML ('mnist_784')  ** it will take a minute **

from sklearn.datasets import fetch_openml

X_raw, Y_raw = fetch_openml('mnist_784', version=1, return_X_y=True)

# Store it in a dataframe
X_raw = pd.DataFrame(X_raw)
Y_raw = pd.DataFrame(Y_raw,columns=['label'])

# Check
strlog = "X_raw is a dataframe of %d rows and %d cols."%(X_raw.shape[0], X_raw.shape[1])
print(strlog)
strlog = "Y_raw is a dataframe of %d elements."%(Y_raw.shape[0])
print(strlog)

X_raw is a dataframe of 70000 rows and 784 cols.
Y_raw is a dataframe of 70000 elements.


#### Separar el conjunto de test y validación

In [3]:
# Test set

test_size = 0.1
X_, Y_, X_test, Y_test = my.single_stratified_split(X_raw, Y_raw, \
                                                    test_size=test_size, random_state=seed)

# Validation set

valid_size = 0.2
X_train, Y_train, X_valid, Y_valid = my.single_stratified_split(X_, Y_, \
                                                                test_size=valid_size, random_state=seed)

## Entrenamiento
#### Preprocesado

In [4]:
# Feature engineering, selection and rescaling to [0,1]

from sklearn.preprocessing import MinMaxScaler

theta = 0.3
X = my.mnist_features( pd.DataFrame(X_train), theta=theta ) 
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
Y = Y_train.values.ravel()

#### *Fit*
+ One vs. One (OvO)
+ One vs. Rest (OvR) 

Ambos necesitan de un clasificador "base"

In [5]:
# this cell will take a while !! 

from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.svm import SVC

base_clf = SVC(kernel='rbf', degree=2, gamma=1, random_state = seed) #<- it can be any other one

# Train OvO - SVM

OvO_clf = OneVsOneClassifier(base_clf)  
OvO_clf.fit(X,Y)

# Train OvR - SVM

OvR_clf = OneVsRestClassifier(base_clf)
OvR_clf.fit(X,Y)

# Check

strlog = "OvO produces %d classifiers" %(len(OvO_clf.estimators_)) 
print(strlog)
strlog = "OvR produces %d classifiers" %(len(OvR_clf.estimators_)) 
print(strlog)

OvO produces 45 classifiers
OvR produces 10 classifiers


## Validación
¡ Recordar que a los datos debemos aplicarles el mismo procedimiento que recibieron los de entrenamiento !

In [6]:
# Feature engineering, selection and rescaling to [0,1]

from sklearn.preprocessing import MinMaxScaler

X_pred = my.mnist_features( pd.DataFrame(X_valid), theta=theta ) 
X_pred = scaler.transform(X_pred)
Y_true = Y_valid.values.ravel()

# predict

Y_pred_OvO = OvO_clf.predict(X_pred)
Y_pred_OvR = OvR_clf.predict(X_pred)

In [25]:
# Performance metrics 

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

conf_mat_OvO = confusion_matrix(Y_true,Y_pred_OvO)
hits_OvO = np.trace(conf_mat_OvO)
conf_mat_OvR = confusion_matrix(Y_true,Y_pred_OvR)
hits_OvR = np.trace(conf_mat_OvR)

# Print out
print("\nOvO confusion matrix:\n")
print(conf_mat_OvO)
print("\n")
print( "OvO Hits  = %d"%(hits_OvO) ) 
print( "OvO Fails = %d"%(Y_true.shape[0]-hits_OvO) )
print("\nOvR confusion matrix:\n")
print(conf_mat_OvR)
print( "\n")
print( "OvR Hits  = %d"%(hits_OvR) ) 
print( "OvR Fails = %d"%(Y_true.shape[0]-hits_OvR) )



OvO confusion matrix:

[[ 921    5   80   50    2   47   43    4   84    7]
 [  21 1094   57   52    1    1   32    4  122   34]
 [  49   45  744  150   79   33   75    4   41   38]
 [  14   51  138  848    0   18   14   53  102   47]
 [   3    4   33    0  967    3   79    3    3  133]
 [ 130   32   82  124    5  294   65   88  254   62]
 [  26   52   90    7   71    7  935    0   16   34]
 [  11   14    3   15    9   30    1 1038  114   78]
 [  40   83   28  134    3   64   45   61  708   63]
 [  12   55   40   33  175   10   44   53   59  771]]


OvO Hits  = 8320
OvO Fails = 4280

OvR confusion matrix:

[[ 987    4   69   55    4   27   50    7   33    7]
 [  29 1101   56   62    3   11   61    8   56   31]
 [  73   47  704  172   90   23   76    7   19   47]
 [  27   56  110  899    0   15   23   62   44   49]
 [   4    8   33    0  986    5   95    3    3   91]
 [ 207   46   71  163    7  246   95  129  101   71]
 [  33   53   80   10   92    5  943    0    9   13]
 [  15   19   

## Comentarios
+ Hemos comprobado que, con el método `fit`, OvO genera un clasificador para cada par de etiquetas posible mientras que OvR genera un clasificador por cada etiqueta.
+ Evidentemente el tiempo de computo es superior a un problema con dos clases.
+ Por otro lado, el método `predict` devuelve una sóla clase, independientemente de haber usado OvO ó OvR.
+ En este cuaderno hemos utilizado SVM, pero se podría haber utilizado cualquier otro clasificador base.<br>
  Muchos de ellos tienen una opción para trabajar en problemas multiclase.
     + Por ejemplo, en la ayuda de [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) podemos ver lo siguiente:
     > + **decision_function_shape**{‘ovo’, ‘ovr’}, default=’ovr’ <br>
       Whether to return a one-vs-rest (‘ovr’) decision function of shape (n_samples, n_classes) as all other classifiers, or the original one-vs-one (‘ovo’) decision function of libsvm which has shape (n_samples, n_classes * (n_classes - 1) / 2). However, one-vs-one (‘ovo’) is always used as multi-class strategy.<br>
       The parameter is ignored for binary classification. <br>
     > + **break_ties**, default=False <br>
     If true, decision_function_shape='ovr', and number of classes > 2, predict will break ties according to the confidence values of decision_function; otherwise the first class among the tied classes is returned. Please note that breaking ties comes at a relatively high computational cost compared to a simple predict.
+ La función *Softmax* resuelve mejor el problema multiclase, pero no elimina la tarea de aprender varios clasificadores. Sin embargo, en redes neuronales la usaremos continuamente.